In [1]:
import pandas as pd
from agentic.enhancer import AgenticEnhancer
import json
from termcolor import colored

In [4]:
def agentic_enhancer(original_prompt):
    
    agentic = AgenticEnhancer(original_prompt)
    agentic_answer = agentic.execute_workflow()

    original_prompt = agentic_answer.get("original_prompt")
    original_prompt_answer = agentic_answer.get("original_prompt_answer")
    original_prompt_lin_probs = agentic_answer.get("original_prompt_lin_probs")
    enhanced_prompt = agentic_answer.get("enhanced_prompt")
    final_prompt_answer = agentic_answer.get("final_prompt_answer")
    final_prompt_lin_probs = agentic_answer.get("final_prompt_lin_probs")
    
    # More thoroughly remove markdown formatting
    if enhanced_prompt and isinstance(enhanced_prompt, str):
        # Remove common markdown formatting
        enhanced_prompt = enhanced_prompt.replace('**', '')
        enhanced_prompt = enhanced_prompt.replace('*', '')
        
        # Remove headings (# to ######)
        for i in range(1, 7):
            heading_marker = '#' * i + ' '
            enhanced_prompt = enhanced_prompt.replace(heading_marker, '')
        enhanced_prompt = enhanced_prompt.replace('#', '')
        
        # Remove other common markdown elements
        enhanced_prompt = enhanced_prompt.replace('===', '')
        enhanced_prompt = enhanced_prompt.replace('---', '')
        enhanced_prompt = enhanced_prompt.replace('```', '')
        enhanced_prompt = enhanced_prompt.replace('`', '')
        enhanced_prompt = enhanced_prompt.replace('> ', '')
        
        # Replace markdown list markers with plain text alternatives
        lines = enhanced_prompt.split('\n')
        for i in range(len(lines)):
            # Replace bullet points and numbered lists
            lines[i] = lines[i].replace('- ', '• ')
            lines[i] = lines[i].lstrip('1234567890. ')
        
        # Put it back together
        enhanced_prompt = '\n'.join(lines)
        
        # Clean up any double spaces resulting from replacements
        while '  ' in enhanced_prompt:
            enhanced_prompt = enhanced_prompt.replace('  ', ' ')
        
        # Clean up any extra newlines
        while '\n\n\n' in enhanced_prompt:
            enhanced_prompt = enhanced_prompt.replace('\n\n\n', '\n\n')
        
    final_result = {
        "original_prompt": original_prompt,
        "original_prompt_answer": original_prompt_answer,
        "original_prompt_lin_probs": original_prompt_lin_probs,
        "enhanced_prompt": enhanced_prompt,
        "final_prompt_answer": final_prompt_answer,
        "final_prompt_lin_probs": final_prompt_lin_probs,
    }
    
    return final_result

In [5]:
# with open('data/prompts.json', 'r') as f:
#     original_prompt = json.load(f)

# df = pd.DataFrame(
#     {
#         "original_prompt": original_prompt,
#         "original_prompt_answer": [None]*len(original_prompt),
#         "original_prompt_lin_probs": [None]*len(original_prompt),
#         "enhanced_prompt": [None]*len(original_prompt),
#         "final_prompt_answer": [None]*len(original_prompt),
#         "final_prompt_lin_probs": [None]*len(original_prompt),
#     }
# )

# display(df.head(5))




# Test with a single prompt
test_prompt = "who is the president"
enhancer = AgenticEnhancer(test_prompt)
final_result = enhancer.execute_workflow()

# Create DataFrame with single result
df = pd.DataFrame([{
    "original_prompt": test_prompt,
    "original_prompt_answer": final_result.get('original_prompt_answer', ''),
    "original_prompt_lin_probs": final_result.get('original_prompt_lin_probs', 0.0),
    "enhanced_prompt": final_result.get('enhanced_prompt', ''),
    "final_prompt_answer": final_result.get('final_prompt_answer', ''),
    "final_prompt_lin_probs": final_result.get('final_prompt_lin_probs', 0.0),
}])

# Set display options for better readability
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

# Display the DataFrame
display(df)



Category Determined:  {query="who is the president", category="simple question"}
Rephrased Prompt:  **Prompt:**  
"Please provide the name of the current president of the United States as of October 2023."
Enhanced Prompt:  Role: You are a knowledgeable political analyst tasked with providing accurate and up-to-date information.

User Prompt: 
Please provide the name of the CURRENT PRESIDENT OF THE UNITED STATES as of OCTOBER 2023.

=== PROMPT EVALUATION ===
Score: 32
Score Justification: The prompt is a basic factual question but lacks specificity regarding which president is being referred to, leading to a lower score.

Improvement Suggestions:
1. Specify which country's president is being asked about to enhance clarity.
2. Provide context or timeframe to narrow down the inquiry (e.g., current president, historical context).
3. Indicate the desired format for the response (e.g., name only, brief biography).


,original_prompt,original_prompt_answer,original_prompt_lin_probs,enhanced_prompt,final_prompt_answer,final_prompt_lin_probs
0,who is the president,,0.0,Role: You are a knowledgeable political analyst tasked with providing accurate and up-to-date information.\n\nUser Prompt: \nPlease provide the name of the CURRENT PRESIDENT OF THE UNITED STATES as of OCTOBER 2023.,,0.0


In [ ]:
for i, row in df.iterrows():
    original_prompt = row['original_prompt']
    final_result = agentic_enhancer(original_prompt)
    
    df.at[i, 'original_prompt_answer'] = final_result['original_prompt_answer']
    df.at[i, 'original_prompt_lin_probs'] = final_result['original_prompt_lin_probs']
    df.at[i, 'enhanced_prompt'] = final_result['enhanced_prompt']
    df.at[i, 'final_prompt_answer'] = final_result['final_prompt_answer']
    df.at[i, 'final_prompt_lin_probs'] = final_result['final_prompt_lin_probs']
    
display(df)

In [6]:
df.to_csv('results/enhancement_results_1.csv', index=False)

#### --------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('results/enhancement_results_1.csv').iloc[:, 1:]
df['original_prompt_lin_probs'] = df['original_prompt_lin_probs'].apply(lambda x: round(x, 6))

mask = df['original_prompt_lin_probs'] < 0.20
df.loc[mask, 'original_prompt_lin_probs'] = 0.50

display(df.head(5))

In [ ]:
print(df.describe())

In [ ]:
# 1. Calculate and visualize average improvement percentage
def plot_improvement_distribution():
    # Calculate improvement percentage
    df['improvement_percentage'] = ((df['final_prompt_lin_probs'] - df['original_prompt_lin_probs']) 
                                  / df['original_prompt_lin_probs'] * 100)
    
    plt.figure(figsize=(10, 6))
    sns.kdeplot(data=df, x='improvement_percentage', fill=True, color='skyblue')
    plt.axvline(df['improvement_percentage'].mean(), color='r', linestyle='--', 
                label=f'Mean Improvement: {df["improvement_percentage"].mean():.1f}%')
    
    plt.title('Distribution of Prompt Enhancement Improvements')
    plt.xlabel('Improvement Percentage')
    plt.ylabel('Density')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print(f"Average improvement: {df['improvement_percentage'].mean():.1f}%")
    print(f"Maximum improvement: {df['improvement_percentage'].max():.1f}%")
    print(f"Minimum improvement: {df['improvement_percentage'].min():.1f}%")

# 2. Compare original vs enhanced confidence scores
def plot_confidence_comparison():
    plt.figure(figsize=(12, 6))
    indices = range(len(df))
    width = 0.35
    
    plt.bar(indices, df['original_prompt_lin_probs'], width, 
            label='Original Confidence', color='lightblue')
    plt.bar([i + width for i in indices], df['final_prompt_lin_probs'], width,
            label='Enhanced Confidence', color='darkblue')
    
    plt.xlabel('Prompt Index')
    plt.ylabel('Model Confidence (Linear Probability)')
    plt.title('Original vs Enhanced Prompt Confidence')
    plt.legend()
    plt.xticks([i + width/2 for i in indices], indices)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_improvement_distribution()
plot_confidence_comparison()